In [98]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

import random 

Lectura de .csv limpios

In [99]:
data = pd.read_csv("../../files/HR_RAW_DATA.csv", index_col = 0)

employees = pd.read_csv("../../files/df_exploracion_employees.csv" , index_col = 0)

experience = pd.read_csv("../../files/df_employee_exp.csv")

surveys = pd.read_csv("../../files/surveys.csv" , index_col = 0)

employment = pd.read_csv("transofrmacion_patri" )

salary = pd.read_csv("../../files/salarydetails_w_floats.csv" , index_col = 0)

hours = pd.read_csv("../../files/df_Salary_SH.csv")

Concatenación de archivos

In [100]:
df_limpio = pd.concat([data['employeenumber'], employees, experience, surveys, employment, salary, hours], axis = 1, ignore_index = False)

Homogeneización de columnas

In [101]:
nuevas_columnas = {'employeenumber' : 'EmployeeID' , 'NUMBERCHILDREN' : 'NumberChildren' , 'NUMCOMPANIESWORKED' : 'NumCompaniesWorked', 'TOTALWORKINGYEARS' : 'TotalWorkingYears' , 'YEARSWITHCURRMANAGER' : 'YearsWithCurrManager' , 'WORKLIFEBALANCE' : 'WorkLifeBalance'}

In [102]:
df_limpio.rename(columns = nuevas_columnas, inplace = True)

Limpieza de duplicados

In [103]:
# Encuentra las filas duplicadas, excluyendo la columna 'RemoteWork'
filas_duplicadas = df_limpio.drop(columns=['RemoteWork']).duplicated(keep=False)

# Filtra las filas que tienen valores nulos en 'EmployeeID'
filas_con_nulos = df_limpio['EmployeeID'].isnull()

# Combina las dos condiciones anteriores para obtener las filas que están duplicadas entre sí y tienen valores nulos en 'EmployeeID'
filas_interesantes = filas_duplicadas & filas_con_nulos

# Obtiene los índices de las filas interesantes
indices_interesantes = df_limpio.index[filas_interesantes]

# Imprime los índices encontrados
print(indices_interesantes)

Int64Index([  81,   83,   86,   87,  307,  312,  313,  318,  319,  321,  322,
             325,  332,  334,  341,  356,  359,  364,  457,  465,  468,  873,
             884,  885,  887,  888,  898,  989, 1002, 1009, 1473, 1475, 1478,
            1479, 1485, 1489, 1496, 1499, 1507, 1510, 1514, 1525, 1526, 1528,
            1529, 1539, 1550, 1555, 1556, 1561, 1562, 1564, 1565, 1568, 1575,
            1577, 1584, 1599, 1602, 1607],
           dtype='int64')


In [104]:
df_limpio = df_limpio.drop_duplicates(keep="first")

In [105]:
df_limpio = df_limpio.reset_index()
df_limpio.drop("index", axis=1, inplace=True)

Imputación de nulos para la columna EmployeeID

In [106]:
def asignar_id(valor, contador):
    
    if pd.isnull(valor):
        contador[0] += 1  # Incrementar el contador si el valor es NaN
        id_nulo = f'nan{contador[0]}'  # Devolver el ID con el contador actualizado
        return id_nulo
    
    else:
        return valor.replace(',0' , '')  # Devolver el valor quitándole los decimales

# Reiniciar el contador cada vez que se aplique la función
def aplicar_asignar_id(columna):
    contador = [0]
    return columna.apply(lambda x: asignar_id(x, contador))

# Aplicar la función a la columna 'EmployeeID' y reasignar los resultados
df_limpio['EmployeeID'] = aplicar_asignar_id(df_limpio['EmployeeID'])


In [107]:
df_limpio['EmployeeID']

0          162
1          259
2          319
3         nan1
4         nan2
         ...  
1558    nan410
1559       966
1560       972
1561    nan411
1562    nan412
Name: EmployeeID, Length: 1563, dtype: object

In [108]:
df_limpio.shape[0]

1563